# SI100B Lecture 6 课堂笔记
- **文件来源**：Lec06_B.pdf
- **课程主题**：Functions as Arguments, Higher Order Procedures, Tuples & Lists
- **核心模块**：Python一等函数、Lambda匿名函数、元组（Tuples）、列表（Lists）

## 1. 一等函数核心特性
函数是Python中的「一等对象」，具备4个关键能力（）：
1. 有明确类型（`function`，与int/float/str同级）
2. 可绑定到新变量（赋值语句）
3. 可作为参数传入其他函数
4. 可作为返回值从其他函数输出

In [ ]:
# 示例1：基础高阶函数calc（）
def calc(op, x, y): 
    return op(x, y)  # 调用传入的函数对象op

# 定义基础运算函数
def add(a, b):
    return a + b

def div(a, b):
    if b != 0:
        return a / b
    print("Denominator was 0.")

# 调用：传入函数对象（非函数调用！）
print(calc(add, 2, 3))  # 输出5（对应）
print(calc(div, 2, 0))  # 输出"Denominator was 0." + None（对应）


# 示例2：进阶高阶函数func_c（）
def func_a():
    print('inside func_a')  # 无返回值→默认返回None

def func_b(y):
    print('inside func_b')
    return y

def func_c(f, z):
    print('inside func_c')
    return f(z)

# 调用验证
print(func_a())          # 输出"inside func_a" + None（）
print(5 + func_b(2))     # 输出"inside func_b" + 7（）
print(func_c(func_b, 3)) # 输出"inside func_c"→"inside func_b" + 3（）

In [ ]:
# 示例：make_prod生成乘法函数（）
def make_prod(a):
    # 内层函数g访问外层变量a（闭包特性）
    def g(b):
        return a * b
    return g  # 返回函数对象g（非调用）

# 方式1：分步绑定+调用
doubler = make_prod(2)  # 绑定"乘以2"的函数
val1 = doubler(3)
print(val1)  # 输出6（）

# 方式2：直接链式调用
val2 = make_prod(2)(3)
print(val2)  # 输出6（）

## 2.Lambda 函数：本质与语法
- **定义**：Python 中的匿名函数（无名称），用于定义简单、一次性的逻辑
- **核心语法**：`lambda 参数列表: 表达式`
  - 参数列表：逗号分隔的变量（如`x`、`x, y`）
  - 表达式：计算结果即返回值（**无需`return`关键字**，且只能有一个表达式）
- **与命名函数的对比**：
  | 特性         | Lambda 函数                  | 命名函数（def）              |
  |--------------|------------------------------|------------------------------|
  | 名称         | 无（匿名）                   | 有（如`def is_even(x):`）    |
  | 代码长度     | 单行表达式                   | 可多行代码块                 |
  | 适用场景     | 简单逻辑、临时使用           | 复杂逻辑、重复使用           |
  | 返回值       | 自动返回表达式结果           | 需显式`return`（无则返回None）|

In [ ]:
# 示例1：单个参数的lambda（判断偶数）
is_even_lambda = lambda x: x % 2 == 0  # 绑定到变量（仅为演示，通常直接使用）

# 对比命名函数
def is_even_def(x):
    return x % 2 == 0

# 效果完全一致
print(is_even_lambda(4))  # 输出：True
print(is_even_def(4))     # 输出：True


# 示例2：多个参数的lambda（求和）
sum_lambda = lambda a, b: a + b

print(sum_lambda(3, 5))  # 输出：8
print(sum_lambda("Hello, ", "World!"))  # 输出：Hello, World!（支持字符串拼接）


# 示例3：lambda 不能包含复杂逻辑（仅允许单个表达式）
try:
    # 错误：lambda 内不能有if-else代码块（除非写成表达式形式）
    invalid_lambda = lambda x: 
        if x > 0:
            return x
        else:
            return -x
except SyntaxError as e:
    print("错误原因：", e)  # 报错：invalid syntax


# 示例4：lambda 中使用三元表达式（允许的复杂逻辑形式）
abs_lambda = lambda x: x if x > 0 else -x  # 三元表达式是单个表达式

print(abs_lambda(-5))  # 输出：5
print(abs_lambda(3))   # 输出：3

In [ ]:
# 示例1：替代命名函数（结合apply函数，）
def apply(criteria, n):
    """统计0~n中满足criteria的数字个数"""
    count = 0
    for i in range(n + 1):
        if criteria(i):
            count += 1
    return count

# 用Lambda替代is_even命名函数
print(apply(lambda x: x % 2 == 0, 10))  # 输出6（0/2/4/6/8/10）


# 示例2：嵌套调用（结合do_twice函数，）
def do_twice(n, fn):
    return fn(fn(n))  # 连续应用两次fn

# Lambda定义"平方"操作
print(do_twice(3, lambda x: x**2))  # 输出81（3²=9，9²=81，）

## *apply 函数：作用与设计
- **定义**：接收两个参数——`criteria`（函数）和`n`（数值），用于统计“0~n中满足`criteria`条件的数字个数”
- **本质**：**高阶函数**（Higher-Order Function）——以函数为参数，实现逻辑复用
- **核心价值**：通过传入不同的`criteria`函数（如判断偶数、奇数、能被3整除等），无需修改`apply`本身即可完成不同统计任务

In [ ]:
# 回顾 apply 函数定义（同上，方便对照）
def apply(criteria, n):
    count = 0
    for i in range(n + 1):
        if criteria(i):
            count += 1
    return count


# 优势1：无需定义命名函数，直接传入lambda作为criteria（简化代码）
# 场景1：统计0~10的偶数（等价于is_even命名函数）
print(apply(lambda x: x % 2 == 0, 10))  # 输出：6


# 场景2：统计0~20中能被3整除的数（临时逻辑，无需命名）
print(apply(lambda x: x % 3 == 0, 20))  # 输出：7（0,3,6,9,12,15,18）


# 场景3：统计0~50中大于30的数
print(apply(lambda x: x > 30, 50))  # 输出：20（31到50共20个数）


# 优势2：快速切换逻辑，无需修改apply函数
# 示例：同一组数据（0~10），用不同lambda统计不同条件
print("偶数：", apply(lambda x: x % 2 == 0, 10))          # 6
print("能被4整除：", apply(lambda x: x % 4 == 0, 10))     # 3（0,4,8）
print("平方大于20：", apply(lambda x: x**2 > 20, 10))    # 6（5到10的平方均>20）

In [ ]:
# 传统方式：每次统计都写新循环（代码冗余）
# 统计0~10的偶数
count_even = 0
for i in range(11):
    if i % 2 == 0:
        count_even += 1

# 统计0~10中能被3整除的数
count_div3 = 0
for i in range(11):
    if i % 3 == 0:
        count_div3 += 1

print(count_even, count_div3)  # 输出：6 4


# Lambda + apply 方式：复用apply函数，仅更换条件（简洁高效）
print(apply(lambda x: x % 2 == 0, 10))   # 6（复用apply）
print(apply(lambda x: x % 3 == 0, 10))   # 4（复用apply）


# 结论：当需要对同一范围执行不同判断时，高阶函数+lambda可大幅减少重复代码

# Lambda 与 apply 核心要点
1. **Lambda 函数**：
   - 语法：`lambda 参数: 表达式`（匿名、单行、自动返回结果）
   - 最佳用途：作为临时函数参数传递给高阶函数（如apply）
   - 限制：仅能包含单个表达式，不能有复杂逻辑（如多行代码、循环）

2. **apply 函数**：
   - 本质：高阶函数，接收“判断函数”和“范围上限”，统计符合条件的数字个数
   - 优势：通过更换“判断函数”（如lambda），实现逻辑复用，减少重复代码

3. **结合使用场景**：
   - 当需要对同一组数据执行多种简单判断时（如统计不同条件的数字个数）
   - 替代“为简单逻辑定义命名函数+循环”的冗余模式

## 3. 元组（Tuples）
- **核心特性**：不可变（无法修改元素）、有序索引、元素类型任意（）
- **定义规则**：
  1. 空元组：`()`
  2. 单元素元组：`(2,)`（必须加逗号，否则视为int）
  3. 多元素元组：`(2, "stu", 3)`（逗号分隔，括号可省略）
- **关键提醒**：元组的本质是「逗号」，而非括号（）

In [ ]:
# 1. 定义元组
empty_tuple = ()
single_tuple = (2,)  # 单元素必须加逗号！
hello_tuple = (2, "stu", 3)

# 2. 索引与切片（同字符串，）
print(hello_tuple[0])       # 输出2（正向索引）
print(hello_tuple[-1])      # 输出3（反向索引）
print(hello_tuple[1:3])     # 输出('stu', 3)（左闭右开切片）

# 3. 常用运算（）
print(hello_tuple + (5, 6)) # 输出(2, 'stu', 3, 5, 6)（拼接）
print((1, 2) * 2)           # 输出(1, 2, 1, 2)（重复）
print(len(hello_tuple))     # 输出3（长度）
print(max((3, 5, 0)))       # 输出5（最大值）

# 4. 不可变特性（修改报错，）
try:
    hello_tuple[1] = 4
except TypeError as e:
    print(e)  # 输出"'tuple' object does not support item assignment"

In [ ]:
# 应用1：变量交换（解包，）
x, y = 1, 2
x, y = y, x  # 元组解包：无需临时变量
print(x, y)  # 输出2 1


# 应用2：函数返回多值（）
def quotient_and_remainder(x, y):
    """返回商和余数（自动打包为元组）"""
    q = x // y
    r = x % y
    return q, r  # 等价于return (q, r)

# 解包接收
quot, rem = quotient_and_remainder(10, 3)
print(quot, rem)  # 输出3 1


# 应用3：可变参数（*args，）
def mean(*args):
    """计算任意个数的平均值（args绑定为元组）"""
    tot = sum(args)
    return tot / len(args)

print(mean(1, 2, 3, 4, 5, 6))  # 输出3.5（args=(1,2,3,4,5,6)）

## 4. 列表（Lists）
- **核心特性**：可变（可修改元素）、有序索引、通常存储同类型元素（）
- **定义**：用`[]`包裹，空列表`[]`，示例`[3, 5, 0]`
- **与元组的关键区别**：列表可变（支持元素修改），元组不可变

In [ ]:
# 1. 定义列表
empty_list = []
num_list = [3, 5, 0]
mixed_list = [1, "a", [2, 3]]  # 允许混合类型（不推荐）

# 2. 可变特性：修改元素（）
num_list[2] = 7  # 修改索引2的元素
print(num_list)  # 输出[3, 5, 7]

# 3. 迭代（同元组/字符串，）
total = 0
for v in num_list:
    total += v
print(total)  # 输出15（3+5+7）

In [ ]:
# 需求：返回列表的元素和与乘积（）
def sum_and_prod(L):
    sum_val = sum(L)  # 求和
    prod_val = 1
    for num in L:
        prod_val *= num  # 求积
    return sum_val, prod_val  # 返回元组

# 测试
print(sum_and_prod([1, 2, 3, 4]))  # 输出(10, 24)（1+2+3+4=10；1*2*3*4=24）

## 5. 核心对比表
### 5.1 元组 vs 列表
| 维度       | 元组（Tuple）                | 列表（List）                  |
|------------|------------------------------|-------------------------------|
| 可变性     | 不可变（无法修改元素）       | 可变（可修改/增删元素）       |
| 定义符号   | `()`（空元组/单元素需逗号）  | `[]`                          |
| 适用场景   | 固定数据、多值返回、变量交换 | 动态数据、需增删改的场景      |

### 5.2 一等函数关键能力
| 能力               | 示例                          | 对应课件段落                 |
|--------------------|-------------------------------|------------------------------|
| 函数作为参数       | `calc(add, 2, 3)`             |               |
| 函数作为返回值     | `doubler = make_prod(2)`      |               |
| 匿名函数简化代码   | `lambda x: x**2`              |               |